In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from functools import reduce
from collections import OrderedDict

Import the raw data file into pandas

In [2]:
xls = pd.ExcelFile('inputData/EAR_Raw_2016.xlsx')

Read in each sheet of the raw data file into a dataframe. All sheets contain the Public Water System ID (PWSID) for the corresponding system. This file contains the following raw data:

Sheet 1 contains information about each individual water system and total water usage

Sheet 2 contains month-by-month water production data

Sheet 3 has CCC, Conservation, and Complaint data

Sheet 4 has Source, Rate, Delivery (month-by-month), Quality, Recycled Water, Treatment, and Emergency data for Large Water Systems (LWS)

Sheet 5 has Source, Rate, Delivery (month-by-month), Quality, and Violation data for Small Water Systems (SWS)

In [3]:
data1 = pd.read_excel(xls,'Table1_Final')
data2 = pd.read_excel(xls,'Table2_Final')
data3 = pd.read_excel(xls,'Table3_Final')
data4 = pd.read_excel(xls,'Table4_LWS')
data5 = pd.read_excel(xls,'Table5_SWS')

Concatenate the last two sheets, since each water system only appears once across the two sheets and there are some columns that only apply to large (or to small) water systems.

In [4]:
frames = [data4,data5]
data45 = pd.concat(frames,sort=True)

Merge dataframes into a single dataframe using the PWSID.

In [5]:
dfs = [data1,data2,data3,data45]
dataAll = reduce(lambda left,right: pd.merge(left,right,on='PWSID'),dfs)

Now that we have the data frame containing all of the data, set the PWSID as the index for easier analysis.

In [6]:
dataAll.set_index('PWSID',inplace=True)

Define the columns you want in the output file. Careful: the column names defined here must match the ones used in the analysis loop! Also define dictionaries for storing month information and unit conversions.

In [7]:
colList = ['PWSID','Water.System.Name','Water.System.Classification','Year','Month','Date','Days.In.Month',
           'WATER PRODUCED Water.Units IN UNITS ORIGINALLY REPORTED',
           'WATER PRODUCED Water.Units REVIEWED BY OFFICE OF INFORMATION MANAGEMENT AND ANALYSIS',
           'UNITS ADJUSTED BY OIMA?','Finished.Water.Vol.Type.Revised','WATER PRODUCED FROM GROUNDWATER',
           'WATER PRODUCED FROM SURFACE WATER','FINISHED WATER PURCHASED OR RECEIVED FROM ANOTHER PUBLIC WATER SYSTEM',
           'WATER SOLD TO ANOTHER PUBLIC WATER SYSTEM','Non-Potable Produced Water (EXCLUDING RECYCLING)',
           'RECYCLED WATER PRODUCED',
           'TOTAL POTABLE WATER PRODUCED USING REVISED UNITS',
           'TOTAL POTABLE WATER IN GALLONS',
           'Population Of Service Area','CALCULATED GPCD (Total Potable Produced in gallons per capita day)']
months = {'Jan':31,'Feb':28,'Mar':31,'Apr':30,'May':31,'Jun':30,'Jul':31,'Aug':31,
          'Sep':30,'Oct':31,'Nov':30,'Dec':31}
convToGal = {'G':1,'MG':1000000,'AF':325851,'CCF':748,'HG':100,'DG':10,'CF':7.48}

Now, loop over each PWSID in the raw data and then over each month. This could in principle be expanded to loop over multiple years as well. For each month, make a dictionary using the column lists, and then fill the dictionary accordingly. Add the dictionary to a list of dictionaries, and then make a Pandas DataFrame using this list of dictionaries.

In [8]:
dList=[]
for index,row in dataAll.iterrows():
#for index,row in dataAll.head().iterrows():
    for month,num_days in months.items():
        gwStr = "WP " + month + " GW"
        swStr = "WP " + month + " SW"
        purStr = "WP " + month + " Purchased"
        totStr = "WP " + month + " Total"
        soldStr = "WP " + month + " Sold"
        nonPStr = "WP " + month + " NonPotable"
        recStr = "WP " + month + " Recycled"
        dTemp = OrderedDict.fromkeys(colList)
        dTemp['PWSID'] = index
        dTemp['Water.System.Name'] = row['Water System Name']
        dTemp['Water.System.Classification'] = row['Water System Classification']
        dTemp['Year'] = 2016 #This is hardcoded for now, but this is not a good permanent solution
        dTemp['Month'] = month
        dTemp['Date'] = month + " 1, 2016"
        dTemp['Days.In.Month'] = num_days
        dTemp['WATER PRODUCED Water.Units in UNITS ORIGINALLY REPORTED'] = row['WP Units of Measure']
        dTemp['Finished.Water.Vol.Type.Revised'] = row['WP Volume Type']
        dTemp['WATER PRODUCED FROM GROUNDWATER'] = row[gwStr]
        dTemp['WATER PRODUCED FROM SURFACEWATER'] = row[swStr]
        dTemp['FINISHED WATER PURCHASED OR RECEIVED FROM ANOTHER PUBLIC WATER SYSTEM'] = row[purStr]
        dTemp['WATER SOLD TO ANOTHER PUBLIC WATER SYSTEM'] = row[soldStr]
        dTemp['Non-Potable Produced Water (EXCLUDING RECYCLING)'] = row[nonPStr]
        dTemp['RECYCLED WATER PRODUCED'] = row[recStr]
        dTemp['TOTAL POTABLE WATER PRODUCED USING REVISED UNITS'] = row[totStr] 
        num_gals = 0
        totFl = 0
        try:
            totFl = float(row[totStr])
        except ValueError:
            totFl = 0
        if row['WP Units of Measure'] in convToGal:
            convFactor = convToGal[row['WP Units of Measure']]
            num_gals = totFl*convFactor
        dTemp['TOTAL POTABLE WATER IN GALLONS'] = num_gals
        try:
            num_pop = float(row['Population'])
        except ValueError:
            num_pop = 0
        dTemp['Population Of Service Area'] = num_pop
        gpcd = 0
        if (num_pop > 0 and num_days > 0): 
            gpcd = (num_gals/num_pop)/num_days
        dTemp['CALCULATED GPCD (Total Potable Produced in gallons per capita day)'] = gpcd
        dList.append(dTemp)
dfProd = pd.DataFrame(dList)
dfProd

,PWSID,Water.System.Name,Water.System.Classification,Year,Month,Date,Days.In.Month,WATER PRODUCED Water.Units IN UNITS ORIGINALLY REPORTED,WATER PRODUCED Water.Units REVIEWED BY OFFICE OF INFORMATION MANAGEMENT AND ANALYSIS,UNITS ADJUSTED BY OIMA?,...,FINISHED WATER PURCHASED OR RECEIVED FROM ANOTHER PUBLIC WATER SYSTEM,WATER SOLD TO ANOTHER PUBLIC WATER SYSTEM,Non-Potable Produced Water (EXCLUDING RECYCLING),RECYCLED WATER PRODUCED,TOTAL POTABLE WATER PRODUCED USING REVISED UNITS,TOTAL POTABLE WATER IN GALLONS,Population Of Service Area,CALCULATED GPCD (Total Potable Produced in gallons per capita day),WATER PRODUCED Water.Units in UNITS ORIGINALLY REPORTED,WATER PRODUCED FROM SURFACEWATER
0,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Jan,"Jan 1, 2016",31,None,None,None,...,0,0,0,0,126403,1.264030e+05,50.0,8.155032e+01,G,0
1,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Feb,"Feb 1, 2016",28,None,None,None,...,0,0,0,0,118248,1.182480e+05,50.0,8.446286e+01,G,0
2,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Mar,"Mar 1, 2016",31,None,None,None,...,0,0,0,0,139399,1.393990e+05,50.0,8.993484e+01,G,0
3,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Apr,"Apr 1, 2016",30,None,None,None,...,0,0,0,0,127710,1.277100e+05,50.0,8.514000e+01,G,0
4,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,May,"May 1, 2016",31,None,None,None,...,0,0,0,0,171440,1.714400e+05,50.0,1.106065e+02,G,0
5,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Jun,"Jun 1, 2016",30,None,None,None,...,0,0,0,0,237645,2.376450e+05,50.0,1.584300e+02,G,0
6,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Jul,"Jul 1, 2016",31,None,None,None,...,0,0,0,0,293738,2.937380e+05,50.0,1.895084e+02,G,0
7,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Aug,"Aug 1, 2016",31,None,None,None,...,0,0,0,0,276755,2.767550e+05,50.0,1.785516e+02,G,0
8,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Sep,"Sep 1, 2016",30,None,None,None,...,0,0,0,0,206123,2.061230e+05,50.0,1.374153e+02,G,0
9,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Oct,"Oct 1, 2016",31,None,None,None,...,0,0,0,0,145825,1.458250e+05,50.0,9.408065e+01,G,0


Potential quality control issues:

-Units could be incorrect. This can be addressed by calculating the GPCD and comparing to average values. 

-Numerical values (e.g. population, number of gallons) could be given as strings. 

-Missing values. Since this report is filed each year by all public water systems, we can use the redundancy in the data to your advantage. For instance, if the population of the PWS is not given in 2017, we can use the 2016 value as a reasonable estimate.

Further improvements: the analysis loop and output can be easily manipulated to produce the desired data format. More quality control can be integrated (either within or outside of the main analysis loop). 

Finally, we write the output to csv format.

In [9]:
dfProd.to_csv("outputData/WaterProduced.csv", index=False)